---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [172]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    #df_out = pd.DataFrame({'x': [], 'State' : [], 'RegionName': []})
    df_out = pd.DataFrame()
    with open('university_towns.txt') as f:
        for row in f:
            if 'edit' in row.rstrip():
                state = row.split('[')[0]
            else:
                df_out = df_out.append({'State': state, 'RegionName' : row.split(' (')[0].rstrip()}, ignore_index=True)
    cols = df_out.columns.tolist()[::-1]
    df_out = df_out[cols]
    return df_out

In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df_gdp = pd.read_excel('gdplev.xls', skiprows=5)
    df_gdp = df_gdp.dropna(how='all').dropna(axis=1, how='all').\
    reset_index(drop=True).drop(df_gdp.columns[:3], axis=1).\
    rename(columns = {'Unnamed: 4' : 'Quarter'}).set_index('Quarter')
    df_gdp.drop(df_gdp.index[0:212], inplace=True)
    gdp_last = 0
    gdp_ind_last = df_gdp.index[0]
    flag = 0 
    for gdp_ind in df_gdp.index:
        if df_gdp.loc[gdp_ind,'GDP in billions of chained 2009 dollars.1'] < gdp_last:
            if flag ==0:
                gdp_ind_last = gdp_ind
            flag +=1
            if flag == 2:
                return gdp_ind_last
        else:
            flag=0
        gdp_last = df_gdp.loc[gdp_ind,'GDP in billions of chained 2009 dollars.1']
    return

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df_gdp = pd.read_excel('gdplev.xls', skiprows=5)
    df_gdp = df_gdp.dropna(how='all').dropna(axis=1, how='all').\
    reset_index(drop=True).drop(df_gdp.columns[:3], axis=1).\
    rename(columns = {'Unnamed: 4' : 'Quarter'}).set_index('Quarter')
    df_gdp.drop(df_gdp.index[0:212], inplace=True)
    gdp_last = 0
    flag = 0 
    for gdp_ind in df_gdp[df_gdp.index >= get_recession_start()].index:
        if df_gdp.loc[gdp_ind,'GDP in billions of chained 2009 dollars.1'] > gdp_last:
            flag +=1
            if flag == 2:
                return gdp_ind
        else:
            flag=0
        gdp_last = df_gdp.loc[gdp_ind,'GDP in billions of chained 2009 dollars.1']
    return   

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df_gdp = pd.read_excel('gdplev.xls', skiprows=5)
    df_gdp = df_gdp.dropna(how='all').dropna(axis=1, how='all').\
    reset_index(drop=True).drop(df_gdp.columns[:3], axis=1).\
    rename(columns = {'Unnamed: 4' : 'Quarter'}).set_index('Quarter')
    rec_start = get_recession_start()
    rec_end = get_recession_end()
    return df_gdp[(df_gdp.index >= rec_start) & (df_gdp.index <= rec_end)]['GDP in billions of chained 2009 dollars.1'].idxmin()

In [7]:
get_recession_bottom()

'2009q2'

In [8]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    first_col = df.columns.get_loc('2000-01')
    #last_col = len(df.columns)-1
    #def get_quartmean(row):
    #    c1 = 0
    #    year = 2000
    #    while c1+first_col <= last_col:
    #        year = 2000 + c1//12
    #        quarter = ((c1)//3)%4+1
    #        row[str(year)+'q'+str(quarter)] = row[first_col+c1: first_col+c1+3].mean()
    #        c1+=3
    #    row['State'] = states[row['State']]
    #    return row
    #df = df[:20].apply(get_quartmean, axis=1).set_index(['State','RegionName'])
    c1 = 0
    year = 0 
    while c1+first_col <= 251:
        year = 2000 + c1//12
        quarter = ((c1)//3)%4+1
        df[str(year)+'q'+str(quarter)] = df.ix[:,c1+first_col:c1+first_col+3].mean(axis=1)
        c1+=3
    df['State'] = df['State'].apply(lambda x: states[x])
    df.set_index(['State','RegionName'], inplace=True)
    df.drop(df.columns[:df.columns.get_loc('2000q1')], axis=1, inplace=True)
    return df

In [367]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    from scipy import stats
    rec_start = get_recession_start()
    rec_end = get_recession_end()
    rec_bottom = get_recession_bottom()
    df_housing = convert_housing_data_to_quarters()
    df_uni_towns = get_list_of_university_towns()
    df_uni_towns = df_uni_towns[df_uni_towns.apply(lambda x: tuple(x.values) in df_housing.index, axis=1)]        
    df_uni_towns['Unitown'] = True
    df_housing = df_housing.merge(df_uni_towns, how='outer', left_index=('State','RegionName'), right_on=('State','RegionName'))
    df_housing.Unitown.fillna(value = False,inplace = True)
    df_housing.set_index(['State','RegionName'],inplace=True)
    t_utown_ratio = df_housing[df_housing.Unitown][rec_start].values\
    /df_housing[df_housing.Unitown][rec_bottom].values
    t_no_utown_ratio = df_housing[np.logical_not(df_housing.Unitown)][rec_start].values\
    /df_housing[np.logical_not(df_housing.Unitown)][rec_bottom].values
    t_utown_ratio = t_utown_ratio[np.logical_not(np.isnan(t_utown_ratio))]
    t_no_utown_ratio = t_no_utown_ratio[np.logical_not(np.isnan(t_no_utown_ratio))]
    pvalue = ttest_ind(t_utown_ratio,t_no_utown_ratio).pvalue 
    if ttest_ind(t_utown_ratio,t_no_utown_ratio).pvalue < 0.01:
        return (pvalue < 0.01, pvalue, 'university town')
    else:
        return (pvalue < 0.01, pvalue, 'non-university town')